<a href="https://colab.research.google.com/github/CharlesColgan/MSDS-6350-6373-CODE/blob/master/hw1/hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import functools 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

cad = pd.read_csv('USDCAD=X.csv')
mxn = pd.read_csv('MXN=X.csv')
jpy = pd.read_csv('JPY=X.csv')
sp500 = pd.read_csv('^GSPC.csv')

#keep close prices only, merge by date, rename columns
data = [cad[['Date','Close']], mxn[['Date','Close']], jpy[['Date','Close']], sp500[['Date','Close']]]
data = functools.reduce(lambda left,right: pd.merge(left,right,on=['Date'], how='outer'), data).dropna()
data.columns = ['Date', 'USD/CAD', 'USD/MXN', 'USD/JPY', 'S&P500']
#reverse the data for descending dates
data = data.iloc[::-1]

#X[t]=set of days t and three previous 
prices = data[data.columns[1:]] #closingPrices
X = pd.DataFrame(columns = ['USDCAD(t)','USDMXN(t)','USDJPY(t)','S&P500(t)',
                            'USDCAD(t-1)','USDMXN(t-1)','USDJPY(t-1)','S&P500(t-1)',
                            'USDCAD(t-2)','USDMXN(t-2)','USDJPY(t-2)','S&P500(t-2)',
                            'USDCAD(t-3)','USDMXN(t-3)','USDJPY(t-3)','S&P500(t-3)'])
for i in range(1, prices.shape[0]-4):
    X.loc[i-1] = tuple(prices[i:i+4].stack())
#add date and true price columns
X.insert(0, "Date", data['Date'][1:(prices.shape[0]-4)].reset_index(drop=True), True)
X['truePrice USDJPY(t+1)'] = np.array(data['USD/JPY'][0:len(X)])
#X.to_csv('prices.csv')

######
Xrescaled = StandardScaler().fit_transform( X[X.columns[1:-1]])
pca = PCA(n_components=16)
pca.fit_transform(Xrescaled)
pve = pca.explained_variance_ratio_.cumsum()
h95 = np.where(pve>=0.95)[0][0]+1

#split train/test
Xtrain, Xtest, Ytrain, Ytest = train_test_split(Xrescaled, X['truePrice USDJPY(t+1)'], test_size=0.20, random_state=0)


,Date,USDCAD(t),USDMXN(t),USDJPY(t),S&P500(t),USDCAD(t-1),USDMXN(t-1),USDJPY(t-1),S&P500(t-1),USDCAD(t-2),USDMXN(t-2),USDJPY(t-2),S&P500(t-2),USDCAD(t-3),USDMXN(t-3),USDJPY(t-3),S&P500(t-3),truePrice USDJPY(t+1)
0,2021-01-29,1.28309,20.252251,104.291000,3714.239990,1.28160,20.331530,104.142998,3787.379883,1.26939,19.988300,103.602997,3750.770020,1.27375,20.105169,103.760002,3849.620117,104.685997
1,2021-01-28,1.28160,20.331530,104.142998,3787.379883,1.26939,19.988300,103.602997,3750.770020,1.27375,20.105169,103.760002,3849.620117,1.27229,19.942881,103.777000,3855.360107,104.291000
2,2021-01-27,1.26939,19.988300,103.602997,3750.770020,1.27375,20.105169,103.760002,3849.620117,1.27229,19.942881,103.777000,3855.360107,1.26467,19.737061,103.505997,3841.469971,104.142998
3,2021-01-26,1.27375,20.105169,103.760002,3849.620117,1.27229,19.942881,103.777000,3855.360107,1.26467,19.737061,103.505997,3841.469971,1.26260,19.585011,103.555000,3853.070068,103.602997
4,2021-01-25,1.27229,19.942881,103.777000,3855.360107,1.26467,19.737061,103.505997,3841.469971,1.26260,19.585011,103.555000,3853.070068,1.27227,19.636000,103.897003,3851.850098,103.760002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,2016-02-12,1.39163,18.903240,112.497002,1864.780029,1.39230,19.140539,113.251999,1829.079956,1.38832,18.916000,115.191002,1851.859985,1.39250,18.789230,115.667999,1852.209961,114.321999
1227,2016-02-11,1.39230,19.140539,113.251999,1829.079956,1.38832,18.916000,115.191002,1851.859985,1.39250,18.789230,115.667999,1852.209961,1.38830,18.650261,116.949997,1853.439941,112.497002
1228,2016-02-10,1.38832,18.916000,115.191002,1851.859985,1.39250,18.789230,115.667999,1852.209961,1.38830,18.650261,116.949997,1853.439941,1.37552,18.446329,116.865997,1880.050049,113.251999
1229,2016-02-09,1.39250,18.789230,115.667999,1852.209961,1.38830,18.650261,116.949997,1853.439941,1.37552,18.446329,116.865997,1880.050049,1.37942,18.266140,118.120003,1915.449951,115.191002
